https://www.kernix.com/blog/image-classification-with-a-pre-trained-deep-neural-network_p11

In [1]:
reset -fs

In [2]:
import os
import re
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

//anaconda/envs/wellio/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### First, create a directory for the images to classify:

#### `/Users/carles/Desktop/data/tutorial/images/ok/` for OK images

#### `/Users/carles/Desktop/data/tutorial/images/nok/` for NOK images

### Second, create a directory for the model:

#### `/Users/carles/Desktop/data/tutorial/imagenet`

### Third, fork and clone:

>`https://github.com/carlespoles/models/tree/master/tutorials/image/imagenet`

then, copy the file `classify_image.py` from the above repository into `/Users/carles/Desktop/data/tutorial/imagenet`

### Forth, run the command:

>`python classify_image.py --model_dir /Users/carles/Desktop/data/tutorial/imagenet`

from the directory `/Users/carles/Desktop/data/tutorial/imagenet` which is where the script `classify_image.py` resides.

It will download a file called `inception-2015-12-05.tgz` and unpack it in the mentioned directory.

In [3]:
model_dir = '/Users/carles/Desktop/data/tutorial/imagenet'

In [4]:
def create_graph():
    with gfile.FastGFile(os.path.join(
        model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [5]:
def extract_features(list_images):
    # The number of features correspond to the output of the specified layer below.
    nb_features = 2048
    features = np.empty((len(list_images), nb_features))
    # We don't need to use the labels provided by InceptionV3.
    # labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for ind, image in enumerate(list_images):
            if (ind%100 == 0):
                print('Processing %s...' % (image))
            if not gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)

            try:
                image_data = gfile.FastGFile(image, 'rb').read()
                predictions = sess.run(next_to_last_tensor,
                {'DecodeJpeg/contents:0': image_data})
            except:
                continue
        features[ind,:] = np.squeeze(predictions)
        # We don't need to use the labels provided by InceptionV3.
        # labels.append(re.split('_\d+',image.split('/')[1])[0])

    # return features, labels
    return features

In [6]:
images_ok_dir = '/Users/carles/Desktop/data/tutorial/images/ok/'

In [7]:
list_images_ok = [images_ok_dir+f for f in os.listdir(images_ok_dir) if re.search('jpg|JPG', f)]

In [8]:
images_nok_dir = '/Users/carles/Desktop/data/tutorial/images/nok/'

In [9]:
list_images_nok = [images_nok_dir+f for f in os.listdir(images_nok_dir) if re.search('jpg|JPG', f)]

In [10]:
list_images = list_images_ok + list_images_nok

In [11]:
features = extract_features(list_images)

Processing /Users/carles/Desktop/data/tutorial/images/ok/00001.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00101.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00201.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00301.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00401.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00501.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00601.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00701.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00801.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/00901.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/01001.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/01101.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/01201.jpg...
Processing /Users/carles/Desktop/data/tutorial/images/ok/01301.jpg...
Processing /Users/ca

In [12]:
pickle.dump(features, open('X', 'wb'))

In [13]:
y = np.vstack((np.array([1]*(len(features)/2)).reshape((len(features)/2), 1), np.array([0]*(len(features)/2)).reshape((len(features)/2), 1)))

In [14]:
pickle.dump(y, open('y', 'wb'))